# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,1.94,91,81,1.67,GL,1717963486
1,1,anar darah,32.7587,61.6540,29.67,15,0,7.36,AF,1717963486
2,2,hawaiian paradise park,19.5933,-154.9731,27.24,62,100,3.09,US,1717963487
3,3,christchurch,-43.5333,172.6333,12.49,83,45,1.54,NZ,1717963202
4,4,paita,-5.0892,-81.1144,21.60,70,19,6.81,PE,1717963487


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    color='City',
    size = 'Humidity',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    alpha = 0.7,
    hover_cols = ['City','Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] < 30) &
                                  (city_data_df['Humidity'] >= 40) &
                                  (city_data_df['Humidity'] <= 60) &
                                  (city_data_df['Cloudiness'] >= 40) &
                                  (city_data_df['Cloudiness'] <= 60) &
                                  (city_data_df['Wind Speed'] < 9)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,asentamiento humano nicolas de pierola,-11.9500,-76.7000,12.16,41,52,3.71,PE,1717963522
134,134,happy valley-goose bay,53.3168,-60.3315,23.17,43,40,6.69,CA,1717963361
159,159,fauske,67.2588,15.3918,15.25,51,46,0.11,NO,1717963540
173,173,thunder bay,48.4001,-89.3168,14.99,55,40,1.54,CA,1717963544
245,245,westport,41.1415,-73.3579,24.14,54,40,4.02,US,1717963532
247,247,crucecita,-28.9500,-58.9833,27.01,51,46,3.33,AR,1717963572
261,261,avarua,-21.2078,-159.7750,23.03,53,40,1.54,CK,1717963427
285,285,sussex,41.1834,-74.6663,23.93,58,40,3.60,US,1717963417
447,447,zhezqazghan,47.7833,67.7667,19.32,55,40,5.00,KZ,1717963642
495,495,portland,45.5234,-122.6762,25.09,58,40,1.79,US,1717963570


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
102,102,asentamiento humano nicolas de pierola,-11.9500,-76.7000,12.16,41,52,3.71,PE,1717963522,
134,134,happy valley-goose bay,53.3168,-60.3315,23.17,43,40,6.69,CA,1717963361,
159,159,fauske,67.2588,15.3918,15.25,51,46,0.11,NO,1717963540,
173,173,thunder bay,48.4001,-89.3168,14.99,55,40,1.54,CA,1717963544,
245,245,westport,41.1415,-73.3579,24.14,54,40,4.02,US,1717963532,
247,247,crucecita,-28.9500,-58.9833,27.01,51,46,3.33,AR,1717963572,
261,261,avarua,-21.2078,-159.7750,23.03,53,40,1.54,CK,1717963427,
285,285,sussex,41.1834,-74.6663,23.93,58,40,3.60,US,1717963417,
447,447,zhezqazghan,47.7833,67.7667,19.32,55,40,5.00,KZ,1717963642,
495,495,portland,45.5234,-122.6762,25.09,58,40,1.79,US,1717963570,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}",
    params["bias"] =  f"proximity:{lng},{lat}"   

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
asentamiento humano nicolas de pierola - nearest hotel: Hostal Primavera
happy valley-goose bay - nearest hotel: Hotel North Two
fauske - nearest hotel: Scandic Fauske
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
westport - nearest hotel: Norwalk Inn and Conference Center
crucecita - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
sussex - nearest hotel: Quality Inn
zhezqazghan - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
102,102,asentamiento humano nicolas de pierola,-11.9500,-76.7000,12.16,41,52,3.71,PE,1717963522,Hostal Primavera
134,134,happy valley-goose bay,53.3168,-60.3315,23.17,43,40,6.69,CA,1717963361,Hotel North Two
159,159,fauske,67.2588,15.3918,15.25,51,46,0.11,NO,1717963540,Scandic Fauske
173,173,thunder bay,48.4001,-89.3168,14.99,55,40,1.54,CA,1717963544,Hampton Inn & Suites by Hilton Thunder Bay
245,245,westport,41.1415,-73.3579,24.14,54,40,4.02,US,1717963532,Norwalk Inn and Conference Center
247,247,crucecita,-28.9500,-58.9833,27.01,51,46,3.33,AR,1717963572,No hotel found
261,261,avarua,-21.2078,-159.7750,23.03,53,40,1.54,CK,1717963427,Paradise Inn
285,285,sussex,41.1834,-74.6663,23.93,58,40,3.60,US,1717963417,Quality Inn
447,447,zhezqazghan,47.7833,67.7667,19.32,55,40,5.00,KZ,1717963642,No hotel found
495,495,portland,45.5234,-122.6762,25.09,58,40,1.79,US,1717963570,the Hoxton


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color='City',
    size = 'Max Temp',
    alpha = 0.7,
    hover_cols = ['City','Humidity','Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Humidity,Hotel Name,Country)